In [4]:
import torch as t
import numpy as np

'''Device where we are running'''
#defining the device
if t.cuda.is_available():
    device = t.device("cuda:0") 
    print("Running on the GPU")
else:
    device = t.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [7]:
#Matrices we need
Sx = t.tensor([[0,1.],[1,0]], dtype=t.cfloat).to(device)
Sy = t.tensor([[0,-1j],[1j,0]], dtype=t.cfloat).to(device)
Sz = t.tensor([[1,0],[0,-1]], dtype=t.cfloat).to(device)
Sp = t.tensor([[0,1],[0,0]], dtype=t.cfloat).to(device)
Sm = t.tensor([[0,0],[1,0]], dtype=t.cfloat).to(device)
Id2 = t.tensor([[1,0],[0,1]], dtype=t.cfloat).to(device)

In [10]:
#Function to add terms to Hamiltonians

def add_op(ops, Js, sites, L): 
    l = [Id2 for i in range(L)]
    for i in range(len(sites)): 
        l[sites[i]] = Js[i]*ops[i]
    m = l[0]
    for i in range(1,L): m = t.kron(m,l[i])
    return m

# XY MODEL

The Hamitlonian is 

\begin{equation}
H_{XY} = h\sum_{i} S_i^z + \sum_{i}\left(\frac{J(1+\eta)}{2}S_i^xS_{i+1}^x + \frac{J(1-\eta)}{2}S_i^yS_{i+1}^y\right)
\end{equation}

where $J, \eta, h$ are the interaction strengths, anisotropy, and the transverse field respectively. Note that when $\eta = 1$, the Hamiltonian is an Ising model.

For $\gamma=h=0$, we have that $[H_{XY}, S^z] = 0$ where $S^z = \sum_{i}S_i^z$. When $\gamma=1$, $[H,Z_2]=0$ where $Z_2 = \prod_{i}S^z_i$ 

Other symmetries include the fermionic operators... 

In [28]:
class XY(): 
    def __init__(self,J, n, h, L, bc = 'periodic'): 
        self.J = J
        self.n = n
        self.h = h
        self.L = L
        self.bc = bc
        
        self.Jx = J*(1+n)/2
        self.Jy = J*(1-n)/2
        
        self.get_H()
        
    def get_H(self): 
        self.H = t.zeros((2**self.L, 2**self.L), dtype=t.cfloat).to(device)
        for i in range(self.L): 
            self.H += add_op([Sz],[self.h], [i], self.L)
            if self.bc == 'periodic' or i<self.L-1:
                self.H += add_op([Sx, Sx], [self.Jx, 1], [i,(i+1)//self.L], self.L)
                self.H += add_op([Sy, Sy], [self.Jy, 1], [i,(i+1)//self.L], self.L)
        return
    
    def evolution(self, t): 
        return t.matrix_exp(self.H*t*1j)

# Toric code 
Imagine a 2D grid. Spins live on the edges. We have periodic boundary conditions in both directions (hence toric). Hamiltonian is 
\begin{equation}
H_{TC} = -J\sum_{v}A_v-J\sum_{p}B_p
\end{equation}
where $A_v=\prod_{e\in v}S_e^x$ with $v$ denoting a vertex of the grid, and $e\in v$ means the edge connected to $v$. $B_p=\prod_{e\in p}S^z_e$ where $p$ is a plaquet. 

For all $v,p$, $[H_{TC}, A_v]= [H_{TC}, B_p]=0$. 

In [197]:
def is_it_int_int(t): 
    flag = False
    if abs(t[0]-int(t[0]))>0 and abs(t[1]-int(t[1]))>0: 
        flag = True 
    return flag

def is_it_int(t): 
    flag = False
    if abs(t[0]-int(t[0]))>0 or abs(t[1]-int(t[1]))>0: 
        flag = True 
    return flag

def location(coords, t): 
    for i in range(len(coords)): 
        for j in range(len(coords[0])): 
            if coords[i][j]==t:
                v = [i,j]
                break
    return v

In [337]:
class ToricCode(): 
    def __init__(self, J, Lx, Ly): 
        self.J = J
        self.Lx = Lx
        self.Ly = Ly
        
        self.coords = [[ (i/2,j/2) for j in range(2*self.Lx) if not is_it_int_int((i/2,j/2))] for i in range(2*self.Ly)]
        self.coords_sites = []
        for i in range(len(self.coords)): 
            if i%2==0: self.coords_sites.append(self.coords[i][1::2])
            elif i%2==1: self.coords_sites.append(self.coords[i][:])
        self.coords_vertices = []
        for i in range(len(self.coords)): 
            if i%2==0: self.coords_vertices.append(self.coords[i][0::2])
        
        self.get_As()
        self.get_Bs()
        
        self.H = -self.J*self.As[0]-self.J*self.Bs[0]
        for i in range(1,len(self.As)): self.H+=-self.J*self.As[i]-self.J*self.Bs[i]
            
    def get_As(self):
        self.As = []
        for i in range(self.Lx):
            for j in range(self.Ly):
                vertex = (i,j)
                #print(vertex)
                self.As.append(self.Av(vertex))
        return 
    
    def get_Bs(self): 
        self.Bs = []
        for p in self.coords_sites[1::2]:
            for s in p:
                self.Bs.append(self.Bp(s))
        return 
    
    def Av(self, v): 
        '''v=(x,y) is a vertex'''
        (x,y) = v 
        if not (abs(x-int(x)))==0 and (abs(y-int(y)))==0: raise ValueError( 'not a valid vertex')
        l = [[ Id2 for i in range(self.Lx)] for j in range(2*self.Ly)]
        #sites = [vr,vl,vup,vdown]
        sites = [(x+0.5,y), (x-0.5, y) if x>0.0 else (self.Lx-1+0.5,y), (x,y+0.5),\
                 (x,y-0.5) if y>0.0 else (x, self.Ly-1+0.5)]
        for i in range(len(sites)):
            loc = location(self.coords_sites, sites[i])
            l[loc[0]][loc[1]] = Sx
        m = Id2
        for i in range(2*self.Ly): 
            for j in range(self.Ly): 
                m = t.kron(m,l[i][j])
        return m
    
    def Bp(self, p):
        '''p=(x,y) is a site, the plaquet lives above the site'''
        (x,y) = p
        if (x-int(x))==0.0 and (y-int(y))==0.0: raise ValueError( 'not a valid site')
        l = [[ Id2 for i in range(self.Lx)] for j in range(2*self.Ly)]
        #sites = [p, pur, pul, puu]
        sites = [p, (x+0.5,y+0.5) if x<self.Lx-0.5 and y<self.Ly-0.5 else (0,y+0.5)]
        sites+= [(x-0.5,y+0.5), (x,y+1.0) if y<self.Ly-1 else (x,0.0) ]
        #print(sites)
        for i in range(len(sites)):
            loc = location(self.coords_sites, sites[i])
            l[loc[0]][loc[1]] = Sz
        m = Id2
        for i in range(2*self.Ly): 
            for j in range(self.Ly): 
                m = t.kron(m,l[i][j])
        return m
    
    def evolution(self, t): 
        return t.matrix_exp(self.H*t*1j)

In [344]:
mytoric = ToricCode(1,2,2)

In [359]:
#The commutation relations are satisfied
Acomms = []
for a in mytoric.As:
    Acomms.append(t.allclose(mytoric.H@a-a@mytoric.H, t.tensor(0+0j)))
Bcomms = []
for b in mytoric.Bs:
    Bcomms.append(t.allclose(mytoric.H@b-b@mytoric.H, t.tensor(0+0j)))
print(Acomms, Bcomms)

[True, True, True, True] [True, True, True, True]
